# YOLO Model Training and Evaluation

This notebook demonstrates how to train a YOLO model using the images and labels in the `train` folder, and then evaluate it on the `test` folder. The workflow includes installing dependencies, configuring the dataset, training, and evaluating the model.

## 1. Install Required Packages

Install the necessary packages for YOLO training and evaluation. This includes `ultralytics`, `opencv-python`, and `matplotlib`.

In [1]:
# Install required packages
!pip install ultralytics opencv-python matplotlib --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Import Libraries

Import all required Python libraries for training and evaluation.

In [2]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from glob import glob


In [3]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Using device: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

CUDA available: True
Using device: NVIDIA GeForce RTX 3060


## 3. Set Up Directory Structure

Define the paths to the train and test folders. Verify that the folders exist and contain the expected data.

In [4]:
# Define paths
train_images = 'train/images'
train_labels = 'train/labels'
test_images = 'test/images'
test_labels = 'test/labels'

# Check if directories exist
for path in [train_images, train_labels, test_images, test_labels]:
    if not os.path.exists(path):
        print(f"Warning: {path} does not exist!")
    else:
        print(f"Found: {path} ({len(os.listdir(path))} files)")

Found: train/images (16284 files)
Found: train/labels (16284 files)
Found: test/images (883 files)
Found: test/labels (883 files)


## 4. Configure YOLO Model Training Parameters

Set up the configuration for YOLO training, including model type, epochs, batch size, and the data.yaml file.

In [5]:

# Training parameters
model_type = 'yolov8n.pt'  # You can change to yolov8s.pt, yolov8m.pt, etc.
epochs = 100
batch = 16
patience = 10

## 5. Train YOLO Model Using Train Folder

Run the training process using the images and annotations in the train folder. The trained model weights will be saved automatically.

In [6]:
# Train YOLO model
model = YOLO(model_type)
results = model.train(data='data.yaml', epochs=epochs, batch=batch,patience = patience, imgsz=640)

# Save the best model path
if hasattr(model, 'ckpt_path') and model.ckpt_path:
    best_model_path = model.ckpt_path
elif hasattr(model, 'trainer') and hasattr(model.trainer, 'best'):
    best_model_path = model.trainer.best
else:
    best_model_path = 'runs/detect/train/weights/best.pt'
print(f"Best model saved at: {best_model_path}")

New https://pypi.org/project/ultralytics/8.3.209 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.204  Python-3.12.8 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train19, nbs=64, nms=False, opset=None, op

## 6. Evaluate Model on Test Folder

Load the trained YOLO model and run inference on the test folder. Calculate and display evaluation metrics such as mAP, precision, and recall.

In [9]:
# Evaluate the trained model
# Automatically find the latest best.pt in any train folder
import glob
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

best_weights = glob.glob("runs/detect/train*/weights/best.pt")
if not best_weights:
    raise FileNotFoundError("No best.pt file found in runs/detect/train*/weights/")
# Get the most recently modified best.pt
best_model_path = max(best_weights, key=os.path.getmtime)
print(f"Loading best model from: {best_model_path}")

# Reload the best model
model = YOLO(best_model_path)

# Run validation on the test set
eval_results = model.val(data='data.yaml')

# Print evaluation metrics
metrics = eval_results.results_dict if hasattr(eval_results, 'results_dict') else eval_results
print("Evaluation metrics:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Loading best model from: runs/detect\train19\weights\best.pt
Ultralytics 8.3.204  Python-3.12.8 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Model summary (fused): 72 layers, 3,012,278 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 117.336.9 MB/s, size: 40.8 KB)
val: Scanning C:\Users\My PC\Documents\ai_222_me_4_yolo\test\labels.cache... 883 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 883/883 882.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 56/56 10.4it/s 5.4s0.1s
                   all        883       2324      0.951      0.965       0.98      0.881
        coffee_nescafe         20        135       0.82      0.881      0.895      0.731
         coffee_kopiko         36        181      0.928      0.923      0.975      0.864
Lucky-Me-Pancit-Canton         25         40      0.991          1      0.995      0.922
           Coke-in-can         25      

In [ ]:
# Gradio interface for YOLO inference
import gradio as gr
from PIL import Image
import numpy as np

# Use the same best_model_path as above (make sure the cell above is run first)
def yolo_infer(image):
    # Convert PIL Image to numpy array
    img_np = np.array(image)
    # Run inference
    results = model(img_np)
    # Get class names and confidences
    boxes = results[0].boxes
    if boxes is not None and len(boxes) > 0:
        # Get the class with the highest confidence
        best_idx = int(boxes.conf.argmax())
        class_id = int(boxes.cls[best_idx])
        conf = float(boxes.conf[best_idx])
        class_name = model.names[class_id] if hasattr(model, 'names') else str(class_id)
        label = f"Predicted: {class_name} (conf: {conf:.2f})"
        # Prepare box info for display
        box_info = []
        for i in range(len(boxes)):
            cid = int(boxes.cls[i])
            cname = model.names[cid] if hasattr(model, 'names') else str(cid)
            cconf = float(boxes.conf[i])
            xyxy = boxes.xyxy[i].tolist() if hasattr(boxes, 'xyxy') else []
            box_info.append(f"{cname} (conf: {cconf:.2f}) box: {xyxy}")
        boxes_str = "\n".join(box_info)
    else:
        label = "No object detected"
        boxes_str = "No boxes detected"
    # Draw boxes on image (optional)
    result_img = results[0].plot() if hasattr(results[0], 'plot') else img_np
    return Image.fromarray(result_img), label, boxes_str

gr.Interface(
    fn=yolo_infer,
    inputs=gr.Image(type="pil", label="Drop an image here"),
    outputs=[gr.Image(type="pil", label="Detection Result"), gr.Textbox(label="Prediction"), gr.Textbox(label="Predicted Boxes")],
    title="YOLOv8 Inference Demo",
    description="Drop an image to see the predicted class and bounding boxes.",
    allow_flagging="never"
    ).launch()

c:\Users\My PC\Documents\ai_222_me_4_yolo\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\My PC\Documents\ai_222_me_4_yolo\env\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



0: 480x640 4 555-sardiness, 42.7ms
Speed: 2.3ms preprocess, 42.7ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 555-sardiness, 122.8ms
Speed: 2.8ms preprocess, 122.8ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 Alaska-Milks, 83.3ms
Speed: 3.0ms preprocess, 83.3ms inference, 6.3ms postprocess per image at shape (1, 3, 480, 640)

0: 640x640 (no detections), 173.0ms
Speed: 3.7ms preprocess, 173.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 158.7ms
Speed: 3.8ms preprocess, 158.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 55.9ms
Speed: 2.8ms preprocess, 55.9ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x480 1 double-black, 181.8ms
Speed: 3.9ms preprocess, 181.8